In [ ]:

def request_guilds(headers):
    try:
        request = httpx.get("https://discord.com/api/v9/users/@me/guilds", headers=headers)
        response = request.json()
    except Exception as e:
        print(e)

    return response
def request_channels(headers, guild_id):
    try:
        request = httpx.get(f"https://discord.com/api/v9/guilds/{guild_id}/channels", headers=headers)
        response = request.json()
    except Exception as e:
        print(e)
    return response
def request_messages(headers, channel_id):
    try:
        request = httpx.get(f"https://discord.com/api/v9/channels/{channel_id}/messages", headers=headers)
        response = request.json()
    except Exception as e:
        print(e)
    return response
def get_all_messages_RealTropea(headers,id):
    messaggi = []
    guilds = request_guilds(headers)
    for guild in guilds:
        if guild["id"] == id:
            channels = request_channels(headers, id)
            for channel in channels:
                messages = request_messages(headers, channel["id"])
                for m in messages:
                    contenuto = m["content"]
                    if contenuto == "":
                        with open("log.json", "a") as f:
                            json.dump(m, f, indent=4)
                        continue
                    autore = m["author"]["username"]
                    messaggi.append((autore, contenuto))
    return messaggi

def get_messages_dictionary(messages):
    autori = {}
    for m in messages:
        if m[0] in autori:
            autori[m[0]].append(m[1])
        else:
            autori[m[0]] = [m[1]]
    return autori         
            
            

In [ ]:
#make a dictionary with the author as the key all the messages as the value
real_tropea = "1115924583258783754"
# messaggi = get_all_messages_RealTropea(headers, real_tropea)
channels = request_channels(headers, real_tropea)
channels
generale_messages = request_messages(headers, "1115924583715967028")

# dizionario = get_messages_dictionary(messaggi)

In [ ]:
thread_id = None
for message in generale_messages:
    if message["content"] == "Thread 1":
        thread_id = message["thread"]["id"]
        break
thread_id

In [ ]:
thread_messages = request_messages(headers, thread_id)
thread_messages
with open("thread.json", "w") as f:
    json.dump(thread_messages, f, indent=4)

In [ ]:
print(dizionario)
for k,v in dizionario.items():
    print(f"{k} ha scritto {len(v)} messaggi")

In [ ]:
def write_message(content,channel_id,headers,bot_messages):
    try:
        request = httpx.post(f"https://discord.com/api/v9/channels/{channel_id}/messages", headers=headers, data=json.dumps({"content":content}))
        response = request.json()
        bot_messages.append(response["id"])
        return response["id"], bot_messages
    except Exception as e:
        print(e)
    
def delete_message(channel_id, message_id, headers):
    try:
        request = httpx.delete(f"https://discord.com/api/v9/channels/{channel_id}/messages/{message_id}", headers=headers)
        response = request.json()
        return response
    except Exception as e:
        print(e)

In [ ]:
bot_messages = []

In [ ]:
id,bot_messages = write_message("Porcodeddio il bruscone", "1231232961303810158", headers,bot_messages)
bot_messages

In [ ]:
deleted = delete_message("1231232961303810158", id, headers)
deleted

In [ ]:
def get_content_and_id(headers, channel_id):
    try:
        request = httpx.get(f"https://discord.com/api/v9/channels/{channel_id}/messages", headers=headers)
        response = request.json()
        messages = {}
        for m in response:
            messages[m["id"]] = m["content"]
        return messages
    except Exception as e:
        print(e)
bruscone_messages = get_content_and_id(headers, "1231232961303810158")

In [ ]:
bruscone_messages

In [ ]:
def delete_bot_messages(headers, channel_id, bot_messages):
    for message in bot_messages:
        delete_message(channel_id, message, headers)
delete_bot_messages(headers, "1231232961303810158", bot_messages)